In [514]:
import pandas as pd
import numpy as np
import util

proficiency_path = './data/tx/proficiency/'

# import column maps
map_files = [
#     '2019_CAMPSTAAR1_map.csv'
#     , '2019_CAMPSTAAR2_map.csv'
#     , '2019_CAMPSTAAR4_map.csv'
#     , '2019_CAMPSTAAR5_map.csv'
#     , '2018_CAMPSTAAR1_map.csv'
#     , '2018_CAMPSTAAR2_map.csv'
#     , '2018_CAMPSTAAR4_map.csv'
#     , '2018_CAMPSTAAR5_map.csv'
#     , '2017_CAMPSTAAR1_map.csv'
#     , '2017_CAMPSTAAR2_map.csv'
#     , '2017_CAMPSTAAR4_map.csv'
#     , '2017_CAMPSTAAR5_map.csv'
#     , '2016_CAMPSTAAR1_map.csv'
#     , '2016_CAMPSTAAR4_map.csv'
#     , '2015_CAMPSTAAR1_map.csv'
#     , '2015_CAMPSTAAR4_map.csv'
    '2014_CAMPSTAAR1_map.csv'
    , '2014_CAMPSTAAR4_map.csv'
#     , '2013_CAMPSTAAR1_map.csv'
#     , '2013_CAMPSTAAR4_map.csv'
]

years = [x[:4] for x in map_files]
filenames = [x[5:15] for x in map_files]

df_map = pd.read_csv(proficiency_path + map_files[0])
df_map['year'] = df_map.apply(lambda x: years[0], axis = 1)
df_map['file'] = df_map.apply(lambda x: filenames[0], axis = 1)

for i in range(1, len(map_files)):
    temp = pd.read_csv(proficiency_path + map_files[i])
    temp['year'] = temp.apply(lambda x: years[i], axis = 1)
    temp['file'] = temp.apply(lambda x: filenames[i], axis = 1)
    df_map = df_map.append(temp, ignore_index = True, sort = True)

In [515]:
# import proficiency files
proficiency_files = [
#     '2019_CAMPSTAAR1.txt'
#     , '2019_CAMPSTAAR2.txt'
#     , '2019_CAMPSTAAR4.txt'
#     , '2019_CAMPSTAAR5.txt'
#     , '2018_CAMPSTAAR1.txt'
#     , '2018_CAMPSTAAR2.txt'
#     , '2018_CAMPSTAAR4.txt'
#     , '2018_CAMPSTAAR5.txt'
#     , '2017_CAMPSTAAR1.txt'
#     , '2017_CAMPSTAAR2.txt'
#     , '2017_CAMPSTAAR4.txt'
#     , '2017_CAMPSTAAR5.txt'
#     , '2016_CAMPSTAAR1.txt'
#     , '2016_CAMPSTAAR4.txt'
#     , '2015_CAMPSTAAR1.txt'
#     , '2015_CAMPSTAAR4.txt'
    '2014_CAMPSTAAR1.txt'
    , '2014_CAMPSTAAR4.txt'
#     , '2013_CAMPSTAAR1.txt'
#     , '2013_CAMPSTAAR4.txt'
]

years = [x[:4] for x in proficiency_files]
filenames = [x[5:15] for x in proficiency_files]

df = pd.read_csv(proficiency_path + proficiency_files[0])
value_vars = list(df.columns)
value_vars.remove('CAMPUS')
df = pd.melt(df, id_vars = ['CAMPUS'], value_vars = value_vars, var_name='NAME', value_name='VALUE')
df = df[df['VALUE'] != '.']
df['year'] = df.apply(lambda x: years[0], axis = 1)
df['file'] = df.apply(lambda x: filenames[0], axis = 1)

for i in range(1, len(proficiency_files)):
    temp = pd.read_csv(proficiency_path + proficiency_files[i])
    value_vars = list(temp.columns)
    value_vars.remove('CAMPUS')
    temp = pd.melt(temp, id_vars = ['CAMPUS'], value_vars = value_vars, var_name='NAME', value_name='VALUE')
    temp = temp[temp['VALUE'] != '.']
    temp['year'] = temp.apply(lambda x: years[i], axis = 1)
    temp['file'] = temp.apply(lambda x: filenames[i], axis = 1)
    df = df.append(temp, ignore_index = True, sort = True)

In [516]:
# import school info files
school_files = [
#     '2019_CREF.txt'
#     , '2018_CREF.txt'
#     , '2017_CREF.txt'
#     , '2016_CREF.txt'
#     , '2015_CREF.txt'
    '2014_CREF.txt'
#     , '2013_CREF.txt'
]

years = [x[:4] for x in school_files]

df_school = pd.read_csv(proficiency_path + school_files[0])
df_school['year'] = df_school.apply(lambda x: years[0], axis = 1)

for i in range(1, len(school_files)):
    temp = pd.read_csv(proficiency_path + school_files[i])
    temp['year'] = temp.apply(lambda x: years[i], axis = 1)
    df_school = df_school.append(temp, ignore_index = True, sort = True)

In [517]:
# join column map to proficiency data, deliberately dropping rows we excluded from the column maps
print(df.shape)
df = df.merge(df_map, on = ['year', 'file', 'NAME'])
print(df.shape)

(6518667, 5)
(6518667, 8)


In [518]:
# drop First Administration records
print(df.shape)
df = df[~df.LABEL.str.contains("First Administration")]
print(df.shape)

(6518667, 8)
(6518667, 8)


In [519]:
# drop rate rows
print(df.shape)
df = df[~df.LABEL.str.contains('Rate')]
print(df.shape)

(6518667, 8)
(4345778, 8)


In [520]:
# remove previous year's records, making exception for 2017 data coming from 2018 file
print(df.shape)
def filterYears(row):
    if row['year'] != '2017' and row['LABEL'].find(str(int(row['year']) - 1)) > 0:
        return 1
    if row['year'] == '2017' and row['LABEL'].find(str(int(row['year']) + 1)) > 0:
        return 1
    return 0
df['deleteFlag'] = df.apply(lambda x: filterYears(x), axis = 1)
df = df[df['deleteFlag'] == 0]
del df['deleteFlag']
print(df.shape)

(4345778, 8)
(2300552, 8)


In [521]:
# parse grade level
def parseGrade(row):
    eocIndex = row['LABEL'].find('EOC')
    if row['LABEL'].upper().find('GRADE 3') > 0:
        return 'Grade 3'
    if row['LABEL'].upper().find('GRADE 4') > 0:
        return 'Grade 4'
    if row['LABEL'].upper().find('GRADE 5') > 0:
        return 'Grade 5'
    if row['LABEL'].upper().find('GRADE 6') > 0:
        return 'Grade 6'
    if row['LABEL'].upper().find('GRADE 7') > 0:
        return 'Grade 7'
    if row['LABEL'].upper().find('GRADE 8') > 0:
        return 'Grade 8'
    if eocIndex > 0:
        return row['LABEL'][eocIndex:]
    return 'All Grades'

df['grade'] = df.apply(lambda x: parseGrade(x), axis = 1)
df['grade'] = df['grade'].str.replace(',', '')
df['grade'] = df['grade'].str.replace('Denominator', '')
df['grade'] = df['grade'].str.replace('Numerator', '')
df['grade'] = df['grade'].str.strip()
print(df['grade'].drop_duplicates())

0                     Grade 3
93776                 Grade 4
186518                Grade 5
2153205               Grade 6
2204201               Grade 7
2248823               Grade 8
3376224     EOC Reading/ELA I
3418260    EOC Reading/ELA II
3457060         EOC Algebra I
3520596        EOC US History
3557082           EOC Biology
Name: grade, dtype: object


In [522]:
# parse group
def parseGroup(row):
    if row['LABEL'].upper().find('CONTINUOUS ENROLLEE') > 0:
        return 'Continuous Enrollee'
    if row['LABEL'].upper().find('MOBILE') > 0:
        return 'Mobile'
    if row['LABEL'].upper().find('NON-AT RISK') > 0:
        return 'Non-At Risk'
    if row['LABEL'].upper().find('NON-CATE') > 0:
        return 'Non-CATE'
    if row['LABEL'].upper().find('NON-ELL') > 0 or row['LABEL'].upper().find('NON-EL') > 0:
        return 'Non-ELL'
    if row['LABEL'].upper().find('NON-ECON DISADV') > 0:
        return 'Non-Econ Disadv'
    if row['LABEL'].upper().find('NON-MIGRANT') > 0:
        return 'Non-Migrant'
    if row['LABEL'].upper().find('NON-SPECIAL ED') > 0:
        return 'Non-Special Ed'
    if row['LABEL'].upper().find('FIRST-YEAR-MONITORED-ELL') > 0 or row['LABEL'].upper().find('FIRST-YEAR-MONITORED-EL') > 0:
        return 'First-Year-Monitored-ELL'
    if row['LABEL'].upper().find('SECOND-YEAR-MONITORED-ELL') > 0 or row['LABEL'].upper().find('SECOND-YEAR-MONITORED-EL') > 0:
        return 'Second-Year-Monitored-ELL'
    if row['LABEL'].upper().find('AFRICAN AMERICAN') > 0:
        return 'African American'
    if row['LABEL'].upper().find('ALL STUDENTS') > 0:
        return 'All Students'
    if row['LABEL'].upper().find('AMERICAN INDIAN') > 0:
        return 'American Indian'
    if row['LABEL'].upper().find('ASIAN') > 0:
        return 'Asian'
    if row['LABEL'].upper().find('FEMALE') > 0:
        return 'Female'
    if row['LABEL'].upper().find('MALE') > 0:
        return 'Male'
    if row['LABEL'].upper().find('AT RISK') > 0:
        return 'At Risk'
    if row['LABEL'].upper().find('ECON DISADV') > 0:
        return 'Econ Disadv'
    if row['LABEL'].upper().find('HISPANIC') > 0:
        return 'Hispanic'
    if row['LABEL'].upper().find('PACIFIC ISLANDER') > 0:
        return 'Pacific Islander'
    if row['LABEL'].upper().find('SPECIAL ED') > 0:
        return 'Special Ed'
    if row['LABEL'].upper().find('WHITE') > 0:
        return 'White'
    if row['LABEL'].upper().find('TWO OR MORE RACES') > 0:
        return 'Two or more races'
    if row['LABEL'].upper().find('ELECTIVE-CATE') > 0:
        return 'Elective-CATE'
    if row['LABEL'].upper().find('CATE') > 0:
        return 'CATE'
    if row['LABEL'].upper().find('MIGRANT') > 0:
        return 'Migrant'
    if row['LABEL'].upper().find('ELL') > 0 or row['LABEL'].upper().find('EL,') > 0:
        return 'ELL'
    return row['LABEL']
        
df['group_state'] = df.apply(lambda x: parseGroup(x), axis = 1)
print(df['group_state'].drop_duplicates())

0                       All Students
8720                African American
15380                          White
22956                       Hispanic
31576                American Indian
33392              Two or more races
38470                          Asian
42458               Pacific Islander
43102                         Female
51798                           Male
60504                    Econ Disadv
69192                     Special Ed
77520                        At Risk
86154                            ELL
3856080                      Migrant
3857400                  Non-Migrant
3866120                     Non-CATE
3874840              Non-Econ Disadv
3883244               Non-Special Ed
3891944                  Non-At Risk
3900616                      Non-ELL
3909324     First-Year-Monitored-ELL
3911878    Second-Year-Monitored-ELL
5278473                         CATE
5278485                Elective-CATE
Name: group_state, dtype: object


In [523]:
# parse subject
def parseSubject(row):
    if row['LABEL'].upper().find('MATH') > 0 or row['LABEL'].upper().find('ALGEBRA') > 0 or row['LABEL'].upper().find('GEOMETRY') > 0:
        return 'Math'
    if row['LABEL'].upper().find('READING') > 0 or row['LABEL'].upper().find('ELA') > 0 or row['LABEL'].upper().find('WRITING') > 0 or row['LABEL'].upper().find('ENGLISH') > 0:
        return 'ELA'
    if row['LABEL'].upper().find('SOCIAL STUDIES') > 0 or row['LABEL'].upper().find('HISTORY') > 0 or row['LABEL'].upper().find('GEOGRAPHY') > 0:
        return 'Social Studies'
    if row['LABEL'].upper().find('SCIENCE') > 0 or row['LABEL'].upper().find('BIOLOGY') > 0 or row['LABEL'].upper().find('PHYSICS') > 0 or row['LABEL'].upper().find('CHEMISTRY') > 0:
        return 'Science'
    return row['LABEL']

df['subject'] = df.apply(lambda x: parseSubject(x), axis = 1)
print(df['subject'].drop_duplicates())

0                     ELA
273730               Math
640186            Science
2521399    Social Studies
Name: subject, dtype: object


In [524]:
# split dataset into numerator/denominator values
df_num = df[df.LABEL.str.contains('Numerator')]
print(df_num.shape)
df_denom = df[df.LABEL.str.contains('Denominator')]
print(df_denom.shape)

(1150276, 11)
(1150276, 11)


In [525]:
# parse performance level
def parsePerformance(row):
    if row['LABEL'].upper().find('APPROACHES GRADE LEVEL') > 0:
        return 'Approaches Grade Level'
    if row['LABEL'].upper().find('MASTERS GRADE LEVEL') > 0:
        return 'Masters Grade Level'
    if row['LABEL'].upper().find('MEETS GRADE LEVEL') > 0:
        return 'Meets Grade Level'
    return 'Phase-in 1 Level II or above'

df_num['performance_level'] = df_num.apply(lambda x: parsePerformance(x), axis = 1)
print(df_num['performance_level'].drop_duplicates())

4360    Phase-in 1 Level II or above
Name: performance_level, dtype: object


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [526]:
# drop unneeded columns
df_num = df_num[['CAMPUS', 'VALUE', 'year', 'grade', 'group_state', 'subject', 'performance_level']]
df_num = df_num.rename(columns = {'VALUE': 'num_at_level'})

def createProficientTF(row):
    if row['performance_level'] == 'Approaches Grade Level':
        return 0
    if row['performance_level'] == 'Phase-in 1 Level II or above' or row['performance_level'] == 'Masters Grade Level' or row['performance_level'] == 'Meets Grade Level':
        return 1
df_num['proficient_tf'] = df_num.apply(lambda x: createProficientTF(x), axis = 1)
print(df_num['proficient_tf'].drop_duplicates())

df_denom = df_denom[['CAMPUS', 'VALUE', 'year', 'grade', 'group_state', 'subject']]
df_denom = df_denom.rename(columns = {'VALUE': 'num_tested'})

4360    1
Name: proficient_tf, dtype: int64


In [527]:
# roll up all results by subject/grade/group/year/campus/proficient_tf/performance_level
print(df_num.shape)
grouped_by = ['year', 'CAMPUS', 'subject', 'grade', 'group_state', 'proficient_tf', 'performance_level']
# change suppressed values to 0, so they aren't integers
df_num['num_at_level'] = [int(x) if int(x) >= 0 else 0 for x in df_num['num_at_level']]
df_num = df_num.groupby(grouped_by, as_index = False)['num_at_level'].sum()
print(df_num.shape)

(1150276, 8)
(1035681, 8)


In [528]:
print(df_denom.shape)
grouped_by.remove('proficient_tf')
grouped_by.remove('performance_level')
# change suppressed values to 0, so they aren't integers
df_denom['num_tested'] = [int(x) if int(x) >= 0 else 0 for x in df_denom['num_tested']]
df_denom = df_denom.groupby(grouped_by, as_index = False)['num_tested'].sum()
print(df_denom.shape)

(1150276, 6)
(1035681, 6)


In [529]:
# join num_at_level and num_tested values to each other
print(df_num.shape)
print(df_denom.shape)
df_joined = df_num.merge(df_denom, on = ['CAMPUS', 'year', 'subject', 'grade', 'group_state'], how='inner')
print(df_joined.head())
print(df_joined.shape)

(1035681, 8)
(1035681, 6)
   year   CAMPUS subject              grade       group_state  proficient_tf  \
0  2014  1902001     ELA  EOC Reading/ELA I  African American              1   
1  2014  1902001     ELA  EOC Reading/ELA I      All Students              1   
2  2014  1902001     ELA  EOC Reading/ELA I             Asian              1   
3  2014  1902001     ELA  EOC Reading/ELA I           At Risk              1   
4  2014  1902001     ELA  EOC Reading/ELA I              CATE              1   

              performance_level  num_at_level  num_tested  
0  Phase-in 1 Level II or above             5           5  
1  Phase-in 1 Level II or above            52          66  
2  Phase-in 1 Level II or above             0           0  
3  Phase-in 1 Level II or above            32          46  
4  Phase-in 1 Level II or above            16          20  
(1035681, 9)


In [530]:
# change suppressed values back to 0
def suppressNumAtLevel(row):
    if row['num_at_level'] == 0 and row['num_tested'] == 0:
        return 'N/A'
    return row['num_at_level']

def suppressNumTested(row):
    if row['num_at_level'] == 0 and row['num_tested'] == 0:
        return 'N/A'
    return row['num_tested']

df_joined['num_at_level'] = df_joined.apply(lambda x: suppressNumAtLevel(x), axis = 1)
df_joined['num_tested'] = df_joined.apply(lambda x: suppressNumTested(x), axis = 1)

In [531]:
# calculate pct_at_level
def calc_pct_at_level(row):
    if row['num_at_level'] == 'N/A' or row['num_tested'] == 'N/A':
        return 'N/A'
    return float(row['num_at_level']) / float(row['num_tested'])
df_joined['pct_at_level'] = df_joined.apply(lambda x: calc_pct_at_level(x), axis = 1)

In [532]:
# join campus data file
print(df_joined.shape)
df_school = df_school[['CAMPUS', 'CAMPNAME', 'DISTRICT', 'DISTNAME']]
df_joined = df_joined.merge(df_school, on = 'CAMPUS')
df_joined = df_joined.rename(columns = {
    'CAMPUS': 'school_id',
    'CAMPNAME': 'school',
    'DISTRICT': 'district_id',
    'distname': 'district'
})
print(df_joined.shape)
print(df_joined.head())

(1035681, 10)
(1035681, 13)
   year  school_id subject              grade       group_state  \
0  2014    1902001     ELA  EOC Reading/ELA I  African American   
1  2014    1902001     ELA  EOC Reading/ELA I      All Students   
2  2014    1902001     ELA  EOC Reading/ELA I             Asian   
3  2014    1902001     ELA  EOC Reading/ELA I           At Risk   
4  2014    1902001     ELA  EOC Reading/ELA I              CATE   

   proficient_tf             performance_level num_at_level  num_tested  \
0              1  Phase-in 1 Level II or above            5           5   
1              1  Phase-in 1 Level II or above           52          66   
2              1  Phase-in 1 Level II or above          N/A           0   
3              1  Phase-in 1 Level II or above           32          46   
4              1  Phase-in 1 Level II or above           16          20   

  pct_at_level      school  district_id    DISTNAME  
0            1  CAYUGA H S         1902  CAYUGA ISD  
1     0.78

In [533]:
# export data
# df.to_csv('./data/finalized/tx_proficiency.csv', index=False)